# Models for final dataset

In [48]:
import pandas as pd
Spatial_data=pd.read_excel('Spatial_data.xlsx')

import ast
X_spatial=[]
for i in range(90):
    for j in range(96):
        X_spatial.append(ast.literal_eval(Spatial_data.T[i][j]))

In [122]:
Spectral_data=pd.read_excel('Spectral_data.xlsx')
X_spectral=[]
for i in range(90):
    for j in range(96):
        X_spectral.append(ast.literal_eval(Spectral_data.T[i][j]))
len(X_spectral)

8640

In [50]:
import pysptools
from pysptools import spectro

In [123]:
for i in range(len(X_spectral)):
    pysptools.spectro.convex_hull_removal(X_spectral[i], wavelengths)

In [9]:
Y=[]
for i in range(90):
    for j in range(96):
        Y.append(i)
len(Y)

8640

In [105]:
from sklearn.preprocessing import LabelBinarizer
y_dense = LabelBinarizer().fit_transform(Y)

*Normalization*

*MSC*

In [8]:
def msc(input_data, reference=None):
    ''' Perform Multiplicative scatter correction'''
 
    # mean centre correction
    for i in range(input_data.shape[0]):
        input_data[i,:] -= input_data[i,:].mean()
 
    # Get the reference spectrum. If not given, estimate it from the mean    
    if reference is None:    
        # Calculate mean
        ref = np.mean(input_data, axis=0)
    else:
        ref = reference
 
    # Define a new array and populate it with the corrected data    
    data_msc = np.zeros_like(input_data)
    for i in range(input_data.shape[0]):
        # Run regression
        fit = np.polyfit(ref, input_data[i,:], 1, full=True)
        # Apply correction
        data_msc[i,:] = (input_data[i,:] - fit[0][1]) / fit[0][0]
 
    return (data_msc, ref)

In [9]:
X_spectral=msc(np.array(X_spectral))[0]

*LDA*

In [124]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components=80)
X_spectral = lda.fit_transform(X_spectral,Y)


*Combination of spectral and spatial features*

In [125]:
X_spectral=list(X_spectral)
type(X_spectral)

list

In [126]:
for i in range(len(X_spectral)):
    X_spectral[i]=list(X_spectral[i])

In [127]:
for i in range(len(X_spectral)):
    for j in range(7):
        X_spectral[i].append(X_spatial[i][j])

In [128]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
print(scaler.fit(X_spectral))
X_spectral = scaler.transform(X_spectral)

StandardScaler()


In [129]:
import numpy as np
X=np.array(X_spectral)
Y=np.array(Y)

In [130]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle="True") 

In [78]:
df=pd.DataFrame(y_test, columns=['label'])

In [79]:
df.to_csv('y_test.csv')

# RandomForestClassifier

In [98]:
from sklearn.ensemble import RandomForestClassifier

In [131]:
clf = RandomForestClassifier(n_estimators=500, criterion='entropy')

In [132]:
clf.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=500)

In [133]:
from sklearn import metrics
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.70      0.80        23
           1       0.78      0.64      0.70        11
           2       0.96      0.92      0.94        24
           3       0.61      0.65      0.63        17
           4       0.56      0.45      0.50        22
           5       0.82      0.67      0.74        21
           6       0.62      0.83      0.71        12
           7       0.50      1.00      0.67        13
           8       0.88      1.00      0.93        21
           9       0.75      0.75      0.75        20
          10       0.81      0.94      0.87        18
          11       0.72      0.72      0.72        18
          12       0.89      0.74      0.81        23
          13       0.56      0.86      0.68        21
          14       0.80      0.89      0.84        18
          15       0.95      0.83      0.89        24
          16       0.67      0.78      0.72        18
          17       0.96    

In [46]:
df=pd.DataFrame(y_pred, columns=['y_pred_rfc'])
df.to_csv('y_pred_rfc.csv')

In [26]:
y_pred_prob=clf.predict_proba(X_test)

In [39]:
np.savetxt('y_pred.csv', y_pred_prob, delimiter=',')

# SVM

In [19]:
from sklearn import svm

In [61]:
clf_1 = svm.SVC(kernel='sigmoid', C=1, probability=True).fit(X_train, y_train)

In [62]:
clf_1.score(X_test, y_test)

0.7679398148148148

In [75]:
y_pred_prob_2=clf_1.predict_proba(X_test)

In [76]:
np.savetxt('y_pred_2.csv', y_pred_prob_2, delimiter=',') 

In [63]:
from sklearn import metrics
from sklearn.metrics import classification_report
y_pred_2 = clf_1.predict(X_test)
print(classification_report(y_test, y_pred_2))

              precision    recall  f1-score   support

           0       0.74      0.78      0.76        18
           1       0.76      0.72      0.74        18
           2       0.96      0.96      0.96        24
           3       0.54      0.78      0.64        18
           4       0.59      0.53      0.56        19
           5       0.69      0.60      0.64        15
           6       0.78      0.95      0.86        22
           7       0.83      0.95      0.88        20
           8       0.95      0.83      0.89        24
           9       0.69      0.92      0.79        12
          10       0.88      0.75      0.81        20
          11       0.71      0.80      0.75        15
          12       0.56      0.60      0.58        15
          13       0.54      0.88      0.67        16
          14       0.68      0.68      0.68        25
          15       0.75      0.83      0.79        18
          16       0.85      0.89      0.87        19
          17       0.88    

In [77]:
df=pd.DataFrame(y_pred_2, columns=['y_pred_svm'])
df.to_csv('y_pred_svm.csv')

# CNN

In [90]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv1D, MaxPool1D, Flatten, Dropout,InputLayer, AveragePooling1D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [87]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [88]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

In [113]:
model=Sequential()
model.add(InputLayer(input_shape=(92,1)))
model.add(Conv1D(64, 9,  padding='same',  activation="swish"))
model.add(Conv1D(64, 9,  padding='same',  activation="swish"))


model.add(Conv1D(128, 9,  padding='same',  activation="swish"))
model.add(Conv1D(128, 9,  padding='same',  activation="swish"))


model.add(Conv1D(256, 9,  padding='same', activation="ReLU", strides=2))

model.add(Conv1D(128, 9,  padding='same',  activation="swish"))
model.add(Conv1D(128, 9,  padding='same',  activation="swish"))
model.add(Conv1D(128, 9,  padding='same',  activation="swish"))

model.add(Conv1D(256, 9,  padding='same', activation="ReLU", strides=2))

model.add(Conv1D(256, 9,  padding='same',  activation="swish"))
model.add(Conv1D(256, 9,  padding='same',  activation="swish"))

model.add(Conv1D(512, 9,  padding='same', activation="ReLU", strides=2))

model.add(Conv1D(256, 9,  padding='same',  activation="swish"))
model.add(Conv1D(256, 9,  padding='same',  activation="swish"))

model.add(MaxPool1D(pool_size=2,strides=2, padding='valid'))

model.add(BatchNormalization())
model.add(Flatten())

model.add(Dropout(0.50))
model.add(Dense(2048, activation="swish"))
model.add(Dropout(0.20))
model.add(Dense(1028, activation="swish"))
model.add(Dropout(0.20))
model.add(Dense(512, activation="swish"))

model.add(BatchNormalization())

model.add(Dense(90, activation='softmax'))
model.build()

In [109]:
epochs=400
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

C:\Users\nishu\anaconda3\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [115]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['Accuracy'])

In [116]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('.tmp/checkpoint_1', save_best_only=True, monitor='val_loss', mode='min', save_weights_only=True)

In [118]:
history = model.fit(X_train, y_train, batch_size=80, epochs=400, verbose=2, shuffle=2, callbacks=[ mcp_save], validation_split=0.3)

Epoch 1/100


KeyboardInterrupt: 

In [69]:
model.load_weights(filepath = '.tmp/checkpoint_1')

In [70]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Accuracy: 54.86%


In [71]:
y_pred_prob_3=model.predict(X_test)

In [72]:
y_pred_3=np.argmax(y_pred_prob_3, axis = 1)

In [73]:
np.savetxt('y_pred_3.csv', y_pred_prob_3, delimiter=',') 

In [74]:
df=pd.DataFrame(y_pred_3, columns=['y_pred_cnn'])
df.to_csv('y_pred_cnn.csv')

In [43]:
pip install pysptools

  Created wheel for pysptools: filename=pysptools-0.15.0-py3-none-any.whl size=8133753 sha256=368e401cf95640523c85460a5590644532caa8ce40bf4d58b89fc8b743484fd1
  Stored in directory: c:\users\nishu\appdata\local\pip\cache\wheels\7b\9f\12\f7d4d71846cbe291217b5bb8d24d1a7f5628f38f97aed2eaa0
Successfully built pysptools



In [3]:
import csv
file=open(r'C:\Users\nishu\Downloads\ML Assignment\RGB and VIS-NIR HSI Data for 90 Rice Seed Varieties\wavelengths.csv')
csvreader = csv.reader(file)
wavelength = []
for row in csvreader:
        wavelength.append(row)
wavelength

wavelengths=[]
for i in range(1,len(wavelength)):
    wavelengths.append(float(wavelength[i][1]))
wavelengths

[383.222222,
 385.583333,
 388.027778,
 390.472222,
 392.916667,
 395.361111,
 397.805556,
 400.25,
 402.694444,
 405.138889,
 407.583333,
 410.027778,
 412.472222,
 414.916667,
 417.361111,
 419.805556,
 422.25,
 424.694444,
 427.138889,
 429.583333,
 432.027778,
 434.472222,
 436.916667,
 439.361111,
 441.805556,
 444.25,
 446.694444,
 449.138889,
 451.583333,
 454.027778,
 456.472222,
 458.916667,
 461.361111,
 463.805556,
 466.25,
 468.694444,
 471.138889,
 473.583333,
 476.027778,
 478.472222,
 480.916667,
 483.361111,
 485.805556,
 488.25,
 490.694444,
 493.138889,
 495.583333,
 498.027778,
 500.472222,
 502.916667,
 505.361111,
 507.805556,
 510.25,
 512.694444,
 515.138889,
 517.583333,
 520.027778,
 522.472222,
 524.916667,
 527.361111,
 529.805556,
 532.25,
 534.694444,
 537.138889,
 539.583333,
 542.027778,
 544.472222,
 546.916667,
 549.361111,
 551.805556,
 554.25,
 556.694444,
 559.138889,
 561.583333,
 564.027778,
 566.472222,
 568.916667,
 571.361111,
 573.805556,
 576.